In [47]:
import pandas as pd
import numpy as np
import math
import random
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate as CV
from surprise.model_selection import GridSearchCV as GSCV
from surprise.model_selection import train_test_split as TTS
from surprise.prediction_algorithms import SVD, SVDpp
from surprise.accuracy import rmse


np.seed = 42

Note:   
Try also:     
biased = False   
SVD++ -but probably worse    

## Initial Model
SVD on the value-change data

In [4]:
df = pd.read_csv('change.csv', index_col=0)
df.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,K,KAI,KAMN,KAR,KB,KBH,KBR,KEN,KEP,KEX
1999-11-01,NaN,NaN,0.25,NaN,NaN,NaN,0.87,NaN,NaN,-1.44,...,-1.18,-0.06,-0.44,NaN,NaN,-0.81,NaN,NaN,0.50,-0.31
1999-11-02,NaN,NaN,0.25,NaN,NaN,NaN,2.69,NaN,NaN,0.00,...,-0.82,-0.06,-0.28,NaN,NaN,0.50,NaN,NaN,0.44,-0.37
1999-11-03,NaN,NaN,-0.13,NaN,NaN,NaN,-1.44,NaN,NaN,-0.76,...,-0.56,0.00,0.47,NaN,NaN,-0.12,NaN,NaN,0.63,0.06
1999-11-04,NaN,NaN,0.13,NaN,NaN,NaN,0.81,NaN,NaN,-0.25,...,-1.19,0.06,0.13,NaN,NaN,0.06,NaN,NaN,-0.50,-0.43
1999-11-05,NaN,NaN,0.25,NaN,NaN,NaN,0.07,NaN,NaN,-0.38,...,-0.25,-0.06,0.66,NaN,NaN,1.19,NaN,NaN,-0.38,-0.06


In [5]:
syms = df.columns

In [10]:
syms

Index(['A', 'AA', 'AAIC', 'AAN', 'AAP', 'AAT', 'AB', 'ABB', 'ABBV', 'ABC',
       ...
       'K', 'KAI', 'KAMN', 'KAR', 'KB', 'KBH', 'KBR', 'KEN', 'KEP', 'KEX'],
      dtype='object', length=1310)

In [1]:
def surprisify(df):
    syms = df.columns
    symID = []
    dateID = []
    value = []

    for sym in syms:
        for row in df[sym].iteritems():
            if not math.isnan(row[1]):
                symID.append(sym)
                dateID.append(row[0])
                value.append(row[1])
                
    sur_dict = {'symID': symID,
              'dateID': dateID,
              'value': value
              }
    sur_df = pd.DataFrame(sur_dict)
    return sur_df

In [6]:
sur_df = surprisify(df)

In [7]:
len(sur_df)

4268702

In [62]:
print(max(sur_df.value))
print(min(sur_df.value))

172.01
-151.59000000000017


In [92]:
ma = max(sur_df.value)
mi = min(sur_df.value)
reader = Reader(rating_scale=(mi, ma))
data = Dataset.load_from_df(sur_df[['dateID', 'symID', 'value']], reader)

In [93]:
train, test = TTS(data, test_size = 0.25)
algo = SVD()
algo.fit(train)
preds = algo.test(test)

rmse(preds)

RMSE: 0.9171


0.9171401364198996

In [85]:
sur_df.value.std()

1.0791845595259724

### SVD on percentage change

In [71]:
pc_df = pd.read_csv('percent_change.csv', index_col=0)
pc_df.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,K,KAI,KAMN,KAR,KB,KBH,KBR,KEN,KEP,KEX
1999-11-01,NaN,NaN,4.940711,NaN,NaN,NaN,3.120516,NaN,NaN,-9.517515,...,-2.982811,-0.849858,-4.000000,NaN,NaN,-3.681818,NaN,NaN,3.052503,-1.722222
1999-11-02,NaN,NaN,4.646840,NaN,NaN,NaN,9.356522,NaN,NaN,0.000000,...,-2.136529,-0.849858,-2.583026,NaN,NaN,2.359604,NaN,NaN,2.550725,-2.107062
1999-11-03,NaN,NaN,-2.284710,NaN,NaN,NaN,-4.438964,NaN,NaN,-5.475504,...,-1.493333,0.000000,4.359926,NaN,NaN,-0.551724,NaN,NaN,3.537339,0.349040
1999-11-04,NaN,NaN,2.338129,NaN,NaN,NaN,2.571429,NaN,NaN,-1.886792,...,-3.238095,0.872093,1.155556,NaN,NaN,0.272727,NaN,NaN,-2.547122,-2.484113
1999-11-05,NaN,NaN,4.347826,NaN,NaN,NaN,0.214988,NaN,NaN,-2.923077,...,-0.701656,-0.864553,5.851064,NaN,NaN,5.274823,NaN,NaN,-2.060738,-0.352941


In [72]:
sur_pcdf = surprisify(pc_df)
sur_pcdf.head()

,symID,dateID,value
0,A,1999-11-18,-3.296703
1,A,1999-11-19,-5.961807
2,A,1999-11-22,6.511740
3,A,1999-11-23,-5.294118
4,A,1999-11-24,2.317468


In [91]:
ma = max(sur_pcdf.value)
mi = min(sur_pcdf.value)
reader = Reader(rating_scale=(mi, ma))
data = Dataset.load_from_df(sur_pcdf[['dateID', 'symID', 'value']], reader)
train, test = TTS(data, test_size = 0.25)
algo = SVD()
algo.fit(train)
preds = algo.test(test)
rmse(preds)

RMSE: 2.2489


2.2489349015690774

In [84]:
sur_pcdf.value.std()

2.3396620341091756

In [87]:
algo.pu.shape

(5382, 100)

In [88]:
algo.pu[0]

array([ 0.32724222, -0.17112312, -0.14482898, -0.19676111, -0.61177941,
       -0.04043862, -0.0596063 , -0.04582079, -0.37100417, -0.29101331,
        0.00423587, -0.56929856, -0.14427849,  0.05842542, -0.00468503,
        0.20014201, -0.00575569,  0.06437647, -0.23000374, -0.04599039,
        0.42742323, -0.40292844,  0.18552409,  0.0656216 , -0.12046813,
       -0.19203081, -0.33647796,  0.07588266, -0.03175989, -0.36092148,
        0.05765124, -0.40853459, -0.17551208, -0.27035477, -0.0800951 ,
        0.22687023, -0.18648973, -0.58957369,  0.29201327,  0.22552867,
        0.29367648, -0.15139456,  0.27722866, -0.00545925, -0.38351509,
       -0.11018775,  0.11501291,  0.26009605,  0.05024719, -0.20667209,
        0.12395133,  0.44523993, -0.58493641,  0.12050439,  0.41889912,
        0.20972228,  0.0889681 ,  0.10638516, -0.35311202,  0.20540077,
       -0.26313919,  0.36572982,  0.24663976,  0.44654162,  0.04357456,
       -0.0472553 ,  0.23486802,  0.1160844 ,  0.29720638,  0.34

## Proposition:
Some days are similar to others. I.e. The factors affecting stocks on one day have previously been in a similar circumstance and the stocks will change in a similar manner as how they previously did.

Issue:Performing SVD on the data at the end of the day is too late for making a prediction.   
Solution: Use midday values to make a prediction.     
Issue: May be able to get some midday results later, but currently would need to get a paid for account for the API
   
   

## Testing 

How well can the results at the end of the day be predicted by those at midday?   
The closest I can get, currently, to testing this is by grouping days into pairs and using the first day's change (acting as the midday change) to predict the total day-pair change (acting as whole day change).
The main difference is that the underlying factors are more likely to change over two days than over one day.

In [26]:
test_df = df.copy()
dates = {}
for row in range(len(test_df)-1):
    dates[test_df.iloc[:,0].index[row]] = test_df.iloc[row] + test_df.iloc[row+1]

In [30]:
test_df = pd.DataFrame(dates).transpose()
test_df.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,K,KAI,KAMN,KAR,KB,KBH,KBR,KEN,KEP,KEX
1999-11-01,NaN,NaN,0.50,NaN,NaN,NaN,3.56,NaN,NaN,-1.44,...,-2.00,-0.12,-0.72,NaN,NaN,-0.31,NaN,NaN,0.94,-0.68
1999-11-02,NaN,NaN,0.12,NaN,NaN,NaN,1.25,NaN,NaN,-0.76,...,-1.38,-0.06,0.19,NaN,NaN,0.38,NaN,NaN,1.07,-0.31
1999-11-03,NaN,NaN,0.00,NaN,NaN,NaN,-0.63,NaN,NaN,-1.01,...,-1.75,0.06,0.60,NaN,NaN,-0.06,NaN,NaN,0.13,-0.37
1999-11-04,NaN,NaN,0.38,NaN,NaN,NaN,0.88,NaN,NaN,-0.63,...,-1.44,0.00,0.79,NaN,NaN,1.25,NaN,NaN,-0.88,-0.49
1999-11-05,NaN,NaN,0.06,NaN,NaN,NaN,-0.37,NaN,NaN,0.06,...,0.62,-0.06,0.97,NaN,NaN,1.63,NaN,NaN,-0.69,0.00


In [49]:
len(test_df)*0.2

1076.2

In [52]:
inds = random.sample(list(test_df.index), 1076)

In [54]:
test_df_test = test_df.loc[inds, :]
test_df_test.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,K,KAI,KAMN,KAR,KB,KBH,KBR,KEN,KEP,KEX
2001-07-10,-3.09,NaN,-0.23,NaN,NaN,NaN,-1.62,0.18,NaN,-0.50,...,-0.16,0.02,-0.11,NaN,NaN,0.48,NaN,NaN,0.02,-0.66
2014-02-12,0.52,NaN,0.21,NaN,0.47,0.56,0.74,0.29,0.98,0.45,...,-0.11,1.50,0.73,-0.02,-0.02,0.16,0.59,NaN,0.34,2.09
2018-10-29,-0.28,-0.28,-0.08,NaN,-2.03,0.86,0.17,-0.11,-1.13,1.35,...,1.83,4.64,0.65,0.49,-1.23,0.02,-0.10,-0.02,-0.17,-0.54
2012-02-29,0.24,NaN,-0.34,NaN,-0.41,-0.71,0.31,-0.36,NaN,-0.18,...,-0.38,-0.25,0.42,-0.18,0.25,-0.14,0.09,NaN,-0.01,1.40
2003-04-22,1.15,NaN,0.45,NaN,2.57,NaN,1.01,0.13,NaN,-0.53,...,0.94,0.09,0.05,NaN,-0.60,2.27,NaN,NaN,0.08,0.24


In [56]:
test_df_train = test_df.drop(inds)
test_df_train.head()

,A,AA,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,...,K,KAI,KAMN,KAR,KB,KBH,KBR,KEN,KEP,KEX
1999-11-01,NaN,NaN,0.50,NaN,NaN,NaN,3.56,NaN,NaN,-1.44,...,-2.00,-0.12,-0.72,NaN,NaN,-0.31,NaN,NaN,0.94,-0.68
1999-11-02,NaN,NaN,0.12,NaN,NaN,NaN,1.25,NaN,NaN,-0.76,...,-1.38,-0.06,0.19,NaN,NaN,0.38,NaN,NaN,1.07,-0.31
1999-11-03,NaN,NaN,0.00,NaN,NaN,NaN,-0.63,NaN,NaN,-1.01,...,-1.75,0.06,0.60,NaN,NaN,-0.06,NaN,NaN,0.13,-0.37
1999-11-04,NaN,NaN,0.38,NaN,NaN,NaN,0.88,NaN,NaN,-0.63,...,-1.44,0.00,0.79,NaN,NaN,1.25,NaN,NaN,-0.88,-0.49
1999-11-05,NaN,NaN,0.06,NaN,NaN,NaN,-0.37,NaN,NaN,0.06,...,0.62,-0.06,0.97,NaN,NaN,1.63,NaN,NaN,-0.69,0.00


In [57]:
len(test_df_train)

4305

In [59]:
sur_train = surprisify(test_df_train)
sur_train.head()

,symID,dateID,value
0,A,1999-11-18,-4.06
1,A,1999-11-19,0.13
2,A,1999-11-22,0.44
3,A,1999-11-24,1.24
4,A,1999-11-29,1.32


In [64]:
print(max(sur_train.value))
print(min(sur_train.value))

222.60000000000002
-190.5600000000002


In [65]:
reader = Reader(rating_scale=(-220, 250))
train = Dataset.load_from_df(sur_train[['dateID', 'symID', 'value']], reader)

In [67]:
test_algo = SVD()
test_algo.fit(train)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'